In [20]:
import cv2
import pyzbar.pyzbar as pyzbar

# QR 코드 이미지 로드
img = cv2.imread('qrcode.png')

# QR 코드 인식
decoded_objects = pyzbar.decode(img)
for obj in decoded_objects:
    print("QR 코드 데이터:", obj.data.decode("utf-8"))

QR 코드 데이터: otpauth://totp/guebin%40jbnu.ac.kr?secret=YPNHCXYNSJS7F6SJ&issuer=jbnu.ac.kr


- `qrcode.png` 는 처음 OPT 등록시 보이는 바코드임. 
- 이미 등록된 경우라면 OTP 를 삭제 후 다시 등록하여 얻을 수 있음. 

In [21]:
import pyotp

# Google Authenticator에 등록된 시크릿 키 (QR 코드에서 추출한 Base32 인코딩)
# QR 코드에 포함된 시크릿 키를 사용해야 합니다.
secret_key = "YPNHCXYNSJS7F6SJ"  # 실제 Google Authenticator 시크릿 키 사용
totp = pyotp.TOTP(secret_key)

# 현재 OTP 코드 생성
current_otp = totp.now()
print(f"현재 Google Authenticator OTP 코드: {current_otp}")

현재 Google Authenticator OTP 코드: 390384


## IMAP

In [22]:
import imaplib
import pyotp

# Google Authenticator 시크릿 키 (Base32 인코딩)
secret_key = "YPNHCXYNSJS7F6SJ"
totp = pyotp.TOTP(secret_key)

# OTP 생성
current_otp = totp.now()
print(f"현재 OTP 코드: {current_otp}")

# 사용자 계정 정보
username = "guebin@jbnu.ac.kr"
password = "tlsrbdus12!@"

# 최종 비밀번호 = 비밀번호 + OTP (예제)
combined_password = f"{password}{current_otp}"

# IMAP 서버 연결 및 로그인 시도
imap_server = imaplib.IMAP4_SSL('imap.jbnu.ac.kr')
try:
    imap_server.login(username, combined_password)
    print("로그인 성공!")
    imap_server.select("INBOX")
    status, messages = imap_server.search(None, "ALL")
    print(f"총 메일 수: {len(messages[0].split())}")
    imap_server.logout()
except imaplib.IMAP4.error:
    print("로그인 실패! 비밀번호 또는 OTP가 올바르지 않습니다.")

현재 OTP 코드: 390384
로그인 실패! 비밀번호 또는 OTP가 올바르지 않습니다.


## SMTP

In [23]:
import smtplib
from email.mime.text import MIMEText
import pyotp

# Google Authenticator 시크릿 키 (Base32)
secret_key = "YPNHCXYNSJS7F6SJ"
totp = pyotp.TOTP(secret_key)

# OTP 생성
current_otp = totp.now()
print(f"현재 OTP 코드: {current_otp}")

# 사용자 계정 정보
username = "guebin@jbnu.ac.kr"
password = "tlsrbdus12!@"

# 비밀번호 + OTP 조합
combined_password = f"{password}{current_otp}"

# SMTP 서버 연결
smtp_server = smtplib.SMTP('smtp.jbnu.ac.kr', 587)
smtp_server.starttls()

try:
    smtp_server.login(username, combined_password)
    msg = MIMEText("Google Authenticator 기반 SMTP 테스트 메일입니다.")
    msg["Subject"] = "Google Authenticator Test"
    msg["From"] = username
    msg["To"] = "gbchoi0814@gmail.com"
    
    smtp_server.sendmail(username, "gbchoi0814@gmail.com", msg.as_string())
    print("메일 발송 성공!")
    smtp_server.quit()
except smtplib.SMTPAuthenticationError:
    print("로그인 실패! 비밀번호 또는 OTP가 올바르지 않습니다.")

현재 OTP 코드: 390384


KeyboardInterrupt: 

## Selenium

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyotp
import time
from webdriver_manager.chrome import ChromeDriverManager

# Google Authenticator 시크릿 키
secret_key = "YPNHCXYNSJS7F6SJ"
totp = pyotp.TOTP(secret_key)

# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# WebDriver Manager를 통한 ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# JBNU 메일 로그인 페이지로 이동
driver.get('https://mail.jbnu.ac.kr')

# 명시적 대기 설정
wait = WebDriverWait(driver, 10)

# 사용자 이름 및 비밀번호 입력
wait.until(EC.presence_of_element_located((By.ID, "cid"))).send_keys("guebin@jbnu.ac.kr")
wait.until(EC.presence_of_element_located((By.ID, "cpw"))).send_keys("tlsrbdus12!@")

# 로그인 버튼 클릭
driver.find_element(By.ID, "btnlogin").click()

# 2단계 인증 코드 대기 및 입력
try:
    wait.until(EC.presence_of_element_located((By.ID, "otp_code1")))
    current_otp = totp.now()
    print(f"현재 OTP 코드: {current_otp}")
    
    for idx, digit in enumerate(current_otp):
        driver.find_element(By.ID, f"otp_code{idx + 1}").send_keys(digit)

    driver.find_element(By.ID, "loginBtn").click()
    print("OTP 입력 완료!")
except Exception as e:
    print("OTP 필드가 감지되지 않았습니다. 아마도 2단계 인증이 비활성화된 것으로 보입니다.")

# 로그인 성공 여부 확인
time.sleep(5)
if "로그아웃" in driver.page_source:
    print("로그인 성공!")

    # 받은 편지함으로 이동
    driver.get('https://mail.jbnu.ac.kr/webmail/lists?t=1735910040')  # 실제 받은 편지함 URL 확인 필요
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mail_subject")))

    # 메일 제목 가져오기 (최대 5개)
    mail_titles = driver.find_elements(By.CLASS_NAME, "mail_subject")[:5]  

    print("\n--- 최근 5개의 메일 제목 ---")
    for idx, title in enumerate(mail_titles):
        print(f"{idx + 1}. {title.text}")

else:
    print("로그인 실패!")

# 드라이버 종료
driver.quit()

현재 OTP 코드: 500622
OTP 입력 완료!
로그인 성공!

--- 최근 5개의 메일 제목 ---
1. Recommendation Confirmation: Suin Kang
2. The Graduate School - University of Washington : RecommendationSubmitted
3. 2024학년도 글로컬30 연구역량강화 지원사업 신청기한 및 조기 신청 안내
4. RE: [RE]인포커스) 자연과학대학 학과 홍보영상 제작 관련 미팅 일정 안내사항
5. RE: [RE]추천서 작성 요청드립니다


In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyotp
import time
from webdriver_manager.chrome import ChromeDriverManager

# Google Authenticator 시크릿 키
secret_key = "YPNHCXYNSJS7F6SJ"
totp = pyotp.TOTP(secret_key)

# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# WebDriver Manager를 통한 ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# JBNU 메일 로그인 페이지로 이동
driver.get('https://mail.jbnu.ac.kr')

# 명시적 대기 설정
wait = WebDriverWait(driver, 10)

# 사용자 이름 및 비밀번호 입력
wait.until(EC.presence_of_element_located((By.ID, "cid"))).send_keys("guebin@jbnu.ac.kr")
wait.until(EC.presence_of_element_located((By.ID, "cpw"))).send_keys("tlsrbdus12!@")

# 로그인 버튼 클릭
driver.find_element(By.ID, "btnlogin").click()

# 2단계 인증 코드 대기 및 입력
try:
    wait.until(EC.presence_of_element_located((By.ID, "otp_code1")))
    current_otp = totp.now()
    print(f"현재 OTP 코드: {current_otp}")
    
    for idx, digit in enumerate(current_otp):
        driver.find_element(By.ID, f"otp_code{idx + 1}").send_keys(digit)

    driver.find_element(By.ID, "loginBtn").click()
    print("OTP 입력 완료!")
except Exception as e:
    print("OTP 필드가 감지되지 않았습니다. 아마도 2단계 인증이 비활성화된 것으로 보입니다.")


# 로그인 성공 여부 확인
time.sleep(5)
if "로그아웃" in driver.page_source:
    print("로그인 성공!")

    # 읽지 않은 메일함으로 이동
    driver.get('https://zm931.mailplug.com/webmail/lists#%7B%22s_fnum%22%3A%22all_mail%22%2C%22view%22%3A%22%22%2C%22list%22%3Atrue%2C%22type%22%3A%22%22%2C%22t%22%3A1735914218822%2C%22serchWord%22%3A%22%22%2C%22searchKind%22%3A%22all%22%2C%22sch_in_word%22%3A%22%22%2C%22ymd%22%3A%22%22%2C%22page%22%3A%22%22%2C%22sod%22%3A%22%22%2C%22sst%22%3A%22%22%2C%22mailrefresh%22%3A%22%22%2C%22mdl%22%3A%22lists%22%2C%22s_mread%22%3A%22N%22%7D')  # 실제 받은 편지함 URL 확인 필요
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mail_subject")))
    print("받음편지함")

    # 안 읽은 메일 제목을 가져오기 (맨 아래 메일)
    mail_titles = driver.find_elements(By.CLASS_NAME, "mail_subject")
    if mail_titles:
        print(f"맨 아래 메일 제목: {mail_titles[-1].text}")
        # 가장 마지막 메일 클릭
        mail_titles[-1].click()
    else:
        print("안 읽은 메일이 없습니다.")
    # 메일 전달 버튼 클릭
    wait = WebDriverWait(driver, 10) 
    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='전달']"))).click()
    
    # 수신자 이메일 주소 입력 (여기서는 'gbchoi0814@gmail.com'으로 설정)
    wait.until(EC.presence_of_element_located((By.ID, "to"))).send_keys("gbchoi0814@gmail.com")
    
    # 전달 버튼 클릭
    wait.until(EC.presence_of_element_located((By.ID, "sendBtn"))).click()
    print("메일이 성공적으로 전달되었습니다!")
else:
    print("로그인 실패!")

현재 OTP 코드: 418195
OTP 입력 완료!
로그인 성공!
받음편지함
맨 아래 메일 제목: asdfasdf


TimeoutException: Message: 
Stacktrace:
#0 0x60cce582c8fa <unknown>
#1 0x60cce533dd20 <unknown>
#2 0x60cce538ca66 <unknown>
#3 0x60cce538cd01 <unknown>
#4 0x60cce53d2184 <unknown>
#5 0x60cce53b0b1d <unknown>
#6 0x60cce53cf560 <unknown>
#7 0x60cce53b0893 <unknown>
#8 0x60cce537f30d <unknown>
#9 0x60cce538032e <unknown>
#10 0x60cce57f900b <unknown>
#11 0x60cce57fcf97 <unknown>
#12 0x60cce57e571c <unknown>
#13 0x60cce57fdb17 <unknown>
#14 0x60cce57ca6cf <unknown>
#15 0x60cce581b6b8 <unknown>
#16 0x60cce581b880 <unknown>
#17 0x60cce582b776 <unknown>
#18 0x79d38ee94ac3 <unknown>


In [38]:
mail_titles[-1].text

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.204); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x55a345d438fa <unknown>
#1 0x55a345854d20 <unknown>
#2 0x55a34585aae3 <unknown>
#3 0x55a34585cfa8 <unknown>
#4 0x55a34585d033 <unknown>
#5 0x55a34589e232 <unknown>
#6 0x55a3458c7af2 <unknown>
#7 0x55a3458980a8 <unknown>
#8 0x55a3458c7cbe <unknown>
#9 0x55a3458e6560 <unknown>
#10 0x55a3458c7893 <unknown>
#11 0x55a34589630d <unknown>
#12 0x55a34589732e <unknown>
#13 0x55a345d1000b <unknown>
#14 0x55a345d13f97 <unknown>
#15 0x55a345cfc71c <unknown>
#16 0x55a345d14b17 <unknown>
#17 0x55a345ce16cf <unknown>
#18 0x55a345d326b8 <unknown>
#19 0x55a345d32880 <unknown>
#20 0x55a345d42776 <unknown>
#21 0x751588a94ac3 <unknown>
